Practice Exam 2 for FINC 489-506; 689-606 (Quantitative Research Methods in Finance)

This part is to import basic modules.

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas_datareader.data as web
import scipy.stats as stats
yf.pdr_override()

Q1: 5 points) Download the following mutual price data from yahoo finance between 2012.01.02 to 2023.04.05. Then, create a data copy and extract Adjusted Close Price ('Adj Close') from the data set you downloaded. Show the result.


Tickers = [
    'BCSIX', 'CIPMX', 'CIPSX', 'FDGRX', 'LSGRX', 'MPEGX', 'MSEQX', 'NWSAX',
    'POAGX', 'POGRX', 'PGBHX', 'PMBPX', 'PRWAX', 'PRDSX', 'RPMGX', 'VHCOX',
    'VIGAX', 'VUG', 'VRGWX', 'VONG', 'VSPGX', 'VOOG', 'VSGIX', 'VBK', 'WGROX',
    'WAAEX', 'AMRMX', 'DHLYX', 'DODGX', 'HAMVX', 'JVMIX', 'OIEJX', 'LSVQX',
    'MCVRX', 'MEIJX', 'OAKMX', 'SCHD', 'VHYAX', 'VYM', 'VMVAX', 'VOE', 'VRVIX',
    'VMFVX', 'VSMVX', 'VSIAX', 'VBR', 'VVIAX', 'VTV', 'VEVRX', 'MADVX', 'CGDV',
    'LCBOX', 'CDIRX', 'FDVV', 'QDF', 'LVHD', 'LCEYX', 'FSIUX', 'SDY', 'PRDGX',
    'TDVG', 'VIG', 'VDADX', 'VDIGX', 'DLN', 'DON', 'DES', 'XMHQ', 'SPY']

In [6]:
# List of best (active) mutual funds in 2023 picked by the morningstar

Tickers = [
    'BCSIX', 'CIPMX', 'CIPSX', 'FDGRX', 'LSGRX', 'MPEGX', 'MSEQX', 'NWSAX',
    'POAGX', 'POGRX', 'PGBHX', 'PMBPX', 'PRWAX', 'PRDSX', 'RPMGX', 'VHCOX',
    'VIGAX', 'VUG', 'VRGWX', 'VONG', 'VSPGX', 'VOOG', 'VSGIX', 'VBK', 'WGROX',
    'WAAEX', 'AMRMX', 'DHLYX', 'DODGX', 'HAMVX', 'JVMIX', 'OIEJX', 'LSVQX',
    'MCVRX', 'MEIJX', 'OAKMX', 'SCHD', 'VHYAX', 'VYM', 'VMVAX', 'VOE', 'VRVIX',
    'VMFVX', 'VSMVX', 'VSIAX', 'VBR', 'VVIAX', 'VTV', 'VEVRX', 'MADVX', 'CGDV',
    'LCBOX', 'CDIRX', 'FDVV', 'QDF', 'LVHD', 'LCEYX', 'FSIUX', 'SDY', 'PRDGX',
    'TDVG', 'VIG', 'VDADX', 'VDIGX', 'DLN', 'DON', 'DES', 'XMHQ', 'SPY'
]

In [7]:
# Download historical data for the tickers
data0 = yf.download(Tickers, start="2012-01-02", end="2024-04-05")['Adj Close']
data = data0.copy()

[*********************100%%**********************]  69 of 69 completed


Q2: 25 points) Calculate daily returns of each mutual funds Then, create another data set that computes the difference of mutual fund returns and SPY (SP500 mutual funds) returns. In so doing, add a condition such that if there are less than 60 observations, remove the specific mutual funds.

In [8]:
# Calculate daily returns
returns = data.pct_change()

# Calculate the difference between each ticker's return and SPY's return
returns_minus_SPY = returns.subtract(returns['SPY'], axis=0)

# Remove the SPY column from the new dataset
returns_minus_SPY = returns_minus_SPY.drop(columns=['SPY'])

# Filter out funds with fewer than n number of  non-NaN observations n = 60
returns_minus_SPY = returns_minus_SPY.dropna(axis=1, thresh=60)

# Display the first few rows of the new dataset
print(returns_minus_SPY.head())

x = returns_minus_SPY
mu = np.mean(x)
rows, columns = x.shape # This is to get the number of data points (rows in this case)
n = rows

print("Number of observations:", n)

Ticker         AMRMX     BCSIX     CDIRX  CGDV     CIPMX     CIPSX       DES  \
Date                                                                           
2012-01-03       NaN       NaN       NaN   NaN       NaN       NaN       NaN   
2012-01-04 -0.000802 -0.011733 -0.002295   NaN -0.007013 -0.009448 -0.006162   
2012-01-05 -0.002279  0.004868 -0.001935   NaN  0.005549  0.002392  0.002612   
2012-01-06 -0.000103  0.001218 -0.001059   NaN  0.003482 -0.003170  0.000391   
2012-01-09 -0.000124  0.004150 -0.001697   NaN  0.000285 -0.000982 -0.001332   

Ticker         DHLYX       DLN     DODGX  ...     VSIAX     VSMVX     VSPGX  \
Date                                      ...                                 
2012-01-03       NaN       NaN       NaN  ...       NaN       NaN       NaN   
2012-01-04 -0.000260 -0.002779 -0.003012  ... -0.005948 -0.008140 -0.001244   
2012-01-05 -0.000048 -0.000037  0.000034  ...  0.003936  0.004932 -0.000798   
2012-01-06 -0.000683 -0.000847 -0.000401  ..

Q3: 25 points) Create a bootsrap function to compute the mean statistic for each bootstrapped sample allowing replacement. The function should return the list of means from each sample.

In [9]:
# Bootstrap function for mean (time-series average for fund return minus SPY)

def bootstrap_mean(data, n_bootstrap=1000):
    bootstrap_means = []
    for _ in range(n_bootstrap):
        bootstrap_sample = np.random.choice(data[~np.isnan(data)], size=len(data), replace=True) #sampling with replacement for observed data
        bootstrap_means.append(np.mean(bootstrap_sample)) # Each repeat will add a mean based on newly bootstrapped samples
    return bootstrap_means


Q4: 30 points) Perform a bootstrap analysis (1000 samples) for each ticker. In doing so, compute the t-statistics and confidence interval (95 percent).

In [10]:
# Perform bootstrap for each ticker and calculate t-statistics
bootstrap_results = {}
for ticker in returns_minus_SPY.columns:
    bootstrap_means = bootstrap_mean(returns_minus_SPY[ticker])
    mean_estimate = np.mean(bootstrap_means)
    std_error = np.std(bootstrap_means, ddof=1) #standard error = std dev of estimate (i.e., mean in this case)
    t_statistic = mean_estimate / std_error
    conf_interval = stats.norm.interval(0.95, loc=mean_estimate, scale=std_error)
    bootstrap_results[ticker] = {'t_statistic': t_statistic, 'conf_interval': conf_interval}

In [11]:
bootstrap_results

{'AMRMX': {'t_statistic': -2.026612390993152,
  'conf_interval': (-0.0002501251109955151, -4.1816432174902765e-06)},
 'BCSIX': {'t_statistic': -0.4526989230432461,
  'conf_interval': (-0.0003670072410361923, 0.00022928076275864536)},
 'CDIRX': {'t_statistic': -1.8459728061969838,
  'conf_interval': (-0.00020549198558368539, 6.154667001762073e-06)},
 'CGDV': {'t_statistic': 2.3073747904123327,
  'conf_interval': (2.1905758037393856e-05, 0.00026907421728863035)},
 'CIPMX': {'t_statistic': -0.595243047965714,
  'conf_interval': (-0.00019396481474541884, 0.0001035954583227059)},
 'CIPSX': {'t_statistic': -1.0273439607577102,
  'conf_interval': (-0.00035889063402566373, 0.00011204355150835058)},
 'DES': {'t_statistic': -1.044602172902403,
  'conf_interval': (-0.0003788576320115846, 0.00011542159174357145)},
 'DHLYX': {'t_statistic': -1.222486228033569,
  'conf_interval': (-0.0001823469536956679, 4.225575054904714e-05)},
 'DLN': {'t_statistic': -1.7224832556832106,
  'conf_interval': (-0.000

Q5: 15 points) Filter the results for those with t-statistics is greater than 1.96 (5 percent significance).

In [12]:
# Filter for positively significant funds
positively_significant_funds = [ticker for ticker, results in bootstrap_results.items() if results['t_statistic'] > 1.96]

# Print results for positively significant funds
print("Positively significant funds:")
for fund in positively_significant_funds:
    print(f"{fund}: T-statistic = {bootstrap_results[fund]['t_statistic']}, 95% Confidence Interval = {bootstrap_results[fund]['conf_interval']}")

Positively significant funds:
CGDV: T-statistic = 2.3073747904123327, 95% Confidence Interval = (2.1905758037393856e-05, 0.00026907421728863035)
FDGRX: T-statistic = 1.9720902491499919, 95% Confidence Interval = (1.2546213951696798e-06, 0.0004068226718813425)


Q6: Bonus 15 points plus $\alpha$) You instead run regressions of each mutual fund to SPY (i.e., a CAPM like regression) then to compute alphas (constant) term instead of the difference and repeat the above bootstrap analysis.